# BG-NBD ve Gamma-Gamma ile CLTV Tahmini

## Görev 1: Veriyi Hazırlama

### Adım 1: flo_data_20k.csv verisini okuyunuz.

In [1]:
!pip install lifetimes
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [2]:
df_ = pd.read_csv(r"C:\Users\myboz\development\miuul\crm_analytics\FLO_CLTV_Prediction\flo_data_20k.csv")

In [3]:
df = df_.copy()

### Adım 2: Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds fonksiyonlarını tanımlayınız.

In [4]:
def outlier_thresholds(df, var):
    qr1 = df[var].quantile(.01)
    qr3 = df[var].quantile(.99)
    interqr_range = qr3 - qr1
    up_limit = qr3 + 1.5 * interqr_range
    low_limit = qr1 - 1.5 * interqr_range
    return low_limit, up_limit

In [5]:
def replace_with_threshold(df, var):
    low_limit, up_limit = outlier_thresholds(df, var)
    df.loc[(df[var] < low_limit), var] = round(low_limit, 0)
    df.loc[(df[var] > up_limit), var] = round(up_limit, 0)

### Adım 3: "order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline", "customer_value_total_ever_online" değişkenlerinin aykırı değeleri varsa baskılayınız.

In [6]:
replace_with_threshold(df, 'order_num_total_ever_online')
replace_with_threshold(df, 'order_num_total_ever_offline')
replace_with_threshold(df, 'customer_value_total_ever_offline')
replace_with_threshold(df, 'customer_value_total_ever_online')

### Adım 4: Omnichannel müşterilerin hem online hem de offline platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.

In [7]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


In [8]:
df['total_order_num'] = df['order_num_total_ever_online'] + df['order_num_total_ever_offline']
df['total_customer_value'] = df['customer_value_total_ever_online'] + df['customer_value_total_ever_offline']

### Adım 5: Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [10]:
var_date = df.columns.str.contains('date')

In [11]:
df.columns[var_date]

Index(['first_order_date', 'last_order_date', 'last_order_date_online',
       'last_order_date_offline'],
      dtype='object')

In [12]:
df[df.columns[var_date]] = df[df.columns[var_date]].astype('datetime64[D]')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

## Görev 2: CLTV  Veri Yapısının Oluşturulması

### Adım 1: Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız.

In [14]:
df['last_order_date'].max()

Timestamp('2021-05-30 00:00:00')

In [15]:
an_date = dt.datetime(2021, 6, 1)

### Adım 2: customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı bir cltv dataframe'i oluşturunuz. Monetary değeri satın alma başına ortalama değer olarak, recency ve tenure değerleri ise haftalık cinsten ifade edilecek.

In [16]:
cltv = pd.DataFrame()

In [17]:
cltv['customer_id'] = df['master_id']

In [22]:
cltv['recency_cltv_weekly'] = ((df['last_order_date'] - df['first_order_date']).dt.days) / 7

In [25]:
cltv['T_weekly'] = ((an_date - df['first_order_date']).dt.days) / 7

In [26]:
cltv['frequency'] = df['total_order_num']

In [27]:
cltv['monetary_cltv_avg'] = df['total_customer_value'] / df['total_order_num']

In [28]:
cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.0,187.874000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.0,95.883333
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.0,117.064000
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.0,60.985000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.0,104.990000


## Görev 3: BG/NBD, Gamma-Gamma Modellerinin Kurulması ve CLTV'nin hesaplanması.

### Adım 1: BG/NBD modelini fit ediniz.

In [29]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [30]:
bgf.fit(cltv['frequency'], cltv['recency_cltv_weekly'], cltv['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

3 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_3_month olarak cltv dataframe'ine ekleyiniz.

In [31]:
cltv['exp_sales_3_month'] = bgf.conditional_expected_number_of_purchases_up_to_time(3 * 4, cltv['frequency'], cltv['recency_cltv_weekly'], cltv['T_weekly'])

6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_6_month olarak cltv dataframe'ine ekleyiniz.

In [32]:
cltv['exp_sales_6_month'] = bgf.conditional_expected_number_of_purchases_up_to_time(6 * 4, cltv['frequency'], cltv['recency_cltv_weekly'], cltv['T_weekly'])

### Adım 2: Gamma-Gamma modelini fit ediniz. Müşterilerin ortalama bırakacakları değeri tahminleyip exp_average_value olarak cltv dataframe'ine ekleyiniz.

In [33]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

In [35]:
ggf.fit(cltv['frequency'], cltv['monetary_cltv_avg'])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

### Adım 3: 6 aylık CLTV hesaplayınız ve cltv ismiyle dataframe'e ekleyiniz.,

In [37]:
cltv['cltv'] = ggf.customer_lifetime_value(bgf,
                                           cltv['frequency'],
                                           cltv['recency_cltv_weekly'],
                                           cltv['T_weekly'],
                                           cltv['monetary_cltv_avg'],
                                           time = 6, # 6 aylık
                                           freq = 'W',
                                           discount_rate = 0.01)

cltv değeri en yüksek 20 kişiyi gözleyiniz.

In [38]:
cltv.sort_values('cltv', ascending=False).head(20)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.857143,7.857143,4.0,1401.800000,1.094385,2.188769,3327.777040
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.142857,13.142857,11.0,758.085455,1.970108,3.940216,3172.394390
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.714286,101.000000,7.0,1106.467143,0.722238,1.444476,1708.982063
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.285714,74.571429,16.0,501.873750,1.565309,3.130618,1662.613492
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714286,67.285714,52.0,166.224615,4.656138,9.312276,1628.887381
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.428571,34.428571,8.0,601.226250,1.265456,2.530912,1623.812684
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.285714,9.428571,2.0,862.690000,0.793924,1.587847,1538.855906
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.714286,13.000000,17.0,259.865294,2.780689,5.561378,1529.227995
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.571429,58.714286,31.0,228.530000,3.083779,6.167558,1485.819156
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.857143,15.571429,3.0,743.586667,0.871564,1.743128,1422.999674


## Görev 4: CLTV Değerine Göre Segmentlerin Oluşturulması

### Adım 1: 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine ekleyiniz.

In [39]:
cltv['cltv_segment'] = pd.qcut(cltv['cltv'], 4, labels=['D', 'C', 'B', 'A'])

In [40]:
cltv

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.0,187.874000,0.973927,1.947853,395.733234,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.0,95.883333,0.983161,1.966323,199.430693,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.0,117.064000,0.670586,1.341172,170.224184,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.0,60.985000,0.700412,1.400824,98.945526,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.0,104.990000,0.396039,0.792077,95.011659,D
...,...,...,...,...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,41.142857,88.428571,3.0,133.986667,0.485785,0.971569,144.101715,C
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,42.285714,65.285714,2.0,195.235000,0.480429,0.960859,212.440779,B
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,88.714286,89.857143,3.0,210.980000,0.481605,0.963210,224.130773,B
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,98.428571,113.857143,6.0,168.295000,0.610224,1.220448,221.078908,B
